In [8]:
import pandas as pd
import glob
import os
import warnings
warnings.filterwarnings("ignore")


FILE_PATH = os.path.expanduser("~/git/dataport/metadata/electricity_egauge_hours/")
METADATA_PATH = os.path.expanduser("~/git/dataport/metadata/metadata.csv")
HDF_PATH = os.path.expanduser("~/monthly-12-18.h5")


In [9]:
# Dropping days when DST changed

dst_times = []
dst_times.append(('2012-03-11 02:00:00', '2012-03-11 03:00:00'))
dst_times.append(('2012-11-04 01:00:00', '2012-11-04 02:00:00'))
dst_times.append(('2013-03-10 02:00:00', '2013-03-10 03:00:00'))
dst_times.append(('2013-11-03 01:00:00', '2013-11-03 02:00:00'))
dst_times.append(('2014-03-09 02:00:00', '2014-03-09 03:00:00'))
dst_times.append(('2014-11-02 01:00:00', '2014-11-02 02:00:00'))
dst_times.append(('2015-03-08 02:00:00', '2015-03-08 03:00:00'))
dst_times.append(('2015-11-01 01:00:00', '2015-11-01 02:00:00'))
dst_times.append(('2016-03-13 02:00:00', '2016-03-13 03:00:00'))
dst_times.append(('2016-11-06 01:00:00', '2016-11-06 02:00:00'))
dst_times.append(('2017-03-12 02:00:00', '2017-03-12 03:00:00'))
dst_times.append(('2017-11-05 01:00:00', '2017-11-05 02:00:00'))
dst_times.append(('2018-03-11 02:00:00', '2018-03-11 03:00:00'))
dst_times.append(('2018-11-04 01:00:00', '2018-11-04 02:00:00'))

In [10]:
# Feeds to ignore
feed_ignore = ['gen', 'grid']


#list_of_buildings = glob.glob("/Users/nipunbatra/w/*.csv")
files = os.listdir(FILE_PATH)
file_size= {x:os.path.getsize(FILE_PATH+x) for x in  files if '.csv' in x}
file_series = pd.Series(file_size)
#file_series = file_series.drop(METADATA_PATH)
fs = file_series[file_series>1000]

In [11]:
store = pd.HDFStore(HDF_PATH, mode='a', complevel=9, complib='blosc')
count = 0

In [12]:
for building_number_csv in fs.index:
    print ("Done {} of {}".format(count, len(fs)))
    try:
        building_path = os.path.join(FILE_PATH, building_number_csv)
        building_number = int(building_number_csv[:-4])
        if building_number in store.keys():
            continue
        df = pd.read_csv(building_path)
        df.index = pd.to_datetime(df["localhour"])
        df = df.drop("localhour", 1)
        # Dropping feeds
        for feed in feed_ignore:
            if feed in df.columns:
                df = df.drop(feed, 1)

        df = df.mul(1000)

        # Dropping feeds with 0 sum
        cols_to_keep = df.sum()[df.sum()>0].index
        df = df[cols_to_keep]

        # Dropping dataid
        if "dataid" in df.columns:
            df = df.drop('dataid', 1)
        df = df.drop(['Unnamed: 0'], axis=1)


        # Fixing DST issues
        for start, end in dst_times:
            ix_drop = df[start:end].index
            df = df.drop(ix_drop)


        # Assigning local timezone
        df = df.tz_localize('US/Central')

        # Making data float32
        df = df.astype('float32')

        
        # Write in temp HDF5 store
        store.put(str(building_number), df, format='table')
        count = count + 1
    except Exception as e:
        print(e)





Done 0 of 849
Done 1 of 849
Done 2 of 849
Done 3 of 849
Done 4 of 849
Done 5 of 849
Done 6 of 849
Done 7 of 849
Done 8 of 849
Done 9 of 849
Done 10 of 849
Done 11 of 849
Done 12 of 849
Done 13 of 849
Done 14 of 849
Done 15 of 849
Done 16 of 849
Done 17 of 849
Done 18 of 849
Done 19 of 849
Done 20 of 849
Done 21 of 849
Done 22 of 849
Done 23 of 849
Done 24 of 849
Done 25 of 849
Done 26 of 849
Done 27 of 849
Done 28 of 849
Done 29 of 849
Done 30 of 849
Done 31 of 849
Done 32 of 849
Done 33 of 849
Done 34 of 849
Done 35 of 849
Done 36 of 849
Done 37 of 849
Done 38 of 849
Done 39 of 849
Done 40 of 849
Done 41 of 849
Done 42 of 849
Done 43 of 849
Done 44 of 849
Done 45 of 849
Done 46 of 849
Done 47 of 849
Done 48 of 849
Done 49 of 849
Done 50 of 849
Done 51 of 849
Done 52 of 849
Done 53 of 849
Done 54 of 849
Done 55 of 849
Done 56 of 849
Done 57 of 849
Done 58 of 849
Done 59 of 849
Done 60 of 849
Done 61 of 849
Done 62 of 849
Done 63 of 849
Done 64 of 849
Done 65 of 849
Done 66 of 849
Done 

Done 519 of 849
Done 520 of 849
Done 521 of 849
Done 522 of 849
Done 523 of 849
Done 524 of 849
Done 525 of 849
Done 526 of 849
Done 527 of 849
Done 528 of 849
Done 529 of 849
Done 530 of 849
Done 531 of 849
Done 532 of 849
Done 533 of 849
Done 534 of 849
Done 535 of 849
Done 536 of 849
Done 537 of 849
Done 538 of 849
Done 539 of 849
Done 540 of 849
Done 541 of 849
Done 542 of 849
Done 543 of 849
Done 544 of 849
Done 545 of 849
Done 546 of 849
Done 547 of 849
Done 548 of 849
Done 549 of 849
Done 550 of 849
Done 551 of 849
Done 552 of 849
Done 553 of 849
Done 554 of 849
Done 555 of 849
Done 556 of 849
Done 557 of 849
Done 558 of 849
Done 559 of 849
Done 560 of 849
Done 561 of 849
Done 562 of 849
Done 563 of 849
Done 564 of 849
Done 565 of 849
Done 566 of 849
Done 567 of 849
Done 568 of 849
Done 569 of 849
Done 570 of 849
Done 571 of 849
Done 572 of 849
Done 573 of 849
Done 574 of 849
Done 575 of 849
Done 576 of 849
Done 577 of 849
Done 578 of 849
Done 579 of 849
Done 580 of 849
Done 581

In [ ]:
store = pd.HDFStore(os.path.expanduser("../2018_all.h5"), 'r')